In [ ]:
import os
import sys
import math
import argparse
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from sklearn.decomposition import PCA
from typing import Dict, List, Tuple, Iterable, Union, Optional, Set, Sequence, Callable, DefaultDict, Any

# Keras imports
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU, ThresholdedReLU, Lambda, Reshape, LayerNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.layers import SpatialDropout1D, SpatialDropout2D, SpatialDropout3D, add, concatenate
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation, Flatten, LSTM, RepeatVector
from tensorflow.keras.layers import Conv1D, Conv2D, Conv3D, UpSampling1D, UpSampling2D, UpSampling3D, MaxPooling1D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling3D, AveragePooling1D, AveragePooling2D, AveragePooling3D, Layer
from tensorflow.keras.layers import SeparableConv1D, SeparableConv2D, DepthwiseConv2D, Concatenate, Add
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalAveragePooling2D, GlobalAveragePooling3D

%matplotlib inline
import matplotlib.pyplot as plt


# ml4h Imports
from ml4h.TensorMap import TensorMap
from ml4h.arguments import parse_args
from ml4h.models import make_multimodal_multitask_model, train_model_from_generators, make_hidden_layer_model, _conv_layer_from_kind_and_dimension
from ml4h.tensor_generators import TensorGenerator, big_batch_from_minibatch_generator, test_train_valid_tensor_generators
from ml4h.recipes import plot_predictions, infer_hidden_layer_multimodal_multitask


Tensor = tf.Tensor

ACTIVATION_CLASSES = {
    'leaky': LeakyReLU(),
    'prelu': PReLU(),
    'elu': ELU(),
    'thresh_relu': ThresholdedReLU,
}
ACTIVATION_FUNCTIONS = {
    'swish': tf.nn.swish,
    'gelu': tfa.activations.gelu,
    'lisht': tfa.activations.lisht,
    'mish': tfa.activations.mish,
}
NORMALIZATION_CLASSES = {
    'batch_norm': BatchNormalization,
    'layer_norm': LayerNormalization,
    'instance_norm': tfa.layers.InstanceNormalization,
    'poincare_norm': tfa.layers.PoincareNormalize,
}
CONV_REGULARIZATION_CLASSES = {
    # class name -> (dimension -> class)
    'spatial_dropout': {2: SpatialDropout1D, 3: SpatialDropout2D, 4: SpatialDropout3D},
    'dropout': defaultdict(lambda _: Dropout),
}
DENSE_REGULARIZATION_CLASSES = {
    'dropout': Dropout,  # TODO: add l1, l2
}

In [ ]:

def _activation_layer(activation: str) -> Activation:
    return (
        ACTIVATION_CLASSES.get(activation, None)
        or Activation(ACTIVATION_FUNCTIONS.get(activation, None) or activation)
    )


def _normalization_layer(norm: str) -> Layer:
    if not norm:
        return lambda x: x
    return NORMALIZATION_CLASSES[norm]()


def _regularization_layer(dimension: int, regularization_type: str, rate: float):
    if not regularization_type:
        return lambda x: x
    if regularization_type in DENSE_REGULARIZATION_CLASSES:
        return DENSE_REGULARIZATION_CLASSES[regularization_type](rate)
    return CONV_REGULARIZATION_CLASSES[regularization_type][dimension](rate)


def _calc_start_shape(
        num_upsamples: int, output_shape: Tuple[int, ...], upsample_rates: Sequence[int], channels: int,
) -> Tuple[int, ...]:
    """
    Given the number of blocks in the decoder and the upsample rates, return required input shape to get to output shape
    """
    upsample_rates = list(upsample_rates) + [1] * len(output_shape)
    return tuple((shape // rate**num_upsamples for shape, rate in zip(output_shape[:-1], upsample_rates))) + (channels,)


class FlatToStructure:
    """Takes a flat input, applies a dense layer, then restructures to output_shape"""
    def __init__(
            self,
            output_shape: Tuple[int, ...],
            activation: str,
            normalization: str,
    ):
        self.input_shapes = output_shape
        self.dense = Dense(units=int(np.prod(output_shape)))
        self.activation = _activation_layer(activation)
        self.reshape = Reshape(output_shape)
        self.norm = _normalization_layer(normalization)

    def __call__(self, x: Tensor) -> Tensor:
        return self.reshape(self.norm(self.activation(self.dense(x))))


def _conv_layer_from_kind_and_dimension(
        dimension: int, conv_layer_type: str, conv_x: List[int], conv_y: List[int], conv_z: List[int],
) -> Tuple[Layer, List[Tuple[int, ...]]]:
    if dimension == 4 and conv_layer_type == 'conv':
        conv_layer = Conv3D
        kernel = zip(conv_x, conv_y, conv_z)
    elif dimension == 3 and conv_layer_type == 'conv':
        conv_layer = Conv2D
        kernel = zip(conv_x, conv_y)
    elif dimension == 2 and conv_layer_type == 'conv':
        conv_layer = Conv1D
        kernel = zip(conv_x)
    elif dimension == 3 and conv_layer_type == 'separable':
        conv_layer = SeparableConv2D
        kernel = zip(conv_x, conv_y)
    elif dimension == 2 and conv_layer_type == 'separable':
        conv_layer = SeparableConv1D
        kernel = zip(conv_x)
    elif dimension == 3 and conv_layer_type == 'depth':
        conv_layer = DepthwiseConv2D
        kernel = zip(conv_x, conv_y)
    else:
        raise ValueError(f'Unknown convolution type: {conv_layer_type} for dimension: {dimension}')
    return conv_layer, list(kernel)


def _upsampler(dimension, pool_x, pool_y, pool_z):
    if dimension == 4:
        return UpSampling3D(size=(pool_x, pool_y, pool_z))
    elif dimension == 3:
        return UpSampling2D(size=(pool_x, pool_y))
    elif dimension == 2:
        return UpSampling1D(size=pool_x)
    

    
def _one_by_n_kernel(dimension):
    return tuple([1] * (dimension - 1))


class DenseConvolutionalBlock:
    def __init__(
            self,
            *,
            dimension: int,
            block_size: int,
            conv_layer_type: str,
            filters: int,
            conv_x: List[int],
            conv_y: List[int],
            conv_z: List[int],
            activation: str,
            normalization: str,
            regularization: str,
            regularization_rate: float,
    ):
        conv_layer, kernels = _conv_layer_from_kind_and_dimension(dimension, conv_layer_type, conv_x, conv_y, conv_z)
        if isinstance(conv_layer, DepthwiseConv2D):
            self.conv_layers = [conv_layer(kernel_size=kernel, padding='same') for kernel in kernels]
        else:
            self.conv_layers = [conv_layer(filters=filters, kernel_size=kernel, padding='same') for kernel in kernels]
        self.activations = [_activation_layer(activation) for _ in range(block_size)]
        self.normalizations = [_normalization_layer(normalization) for _ in range(block_size)]
        self.regularizations = [_regularization_layer(dimension, regularization, regularization_rate) for _ in range(block_size)]
        print(f'Dense Block Convolutional Layers (num_filters, kernel_size): {list(zip([filters]*len(kernels), kernels))}')

    def __call__(self, x: Tensor) -> Tensor:
        dense_connections = [x]
        for i, (convolve, activate, normalize, regularize) in enumerate(
            zip(
                    self.conv_layers, self.activations, self.normalizations, self.regularizations,
            ),
        ):
            x = normalize(regularize(activate(convolve(x))))
            if i < len(self.conv_layers) - 1:  # output of block does not get concatenated to
                dense_connections.append(x)
                x = Concatenate()(dense_connections[:])  # [:] is necessary because of tf weirdness
        return x

    
class ConvDecoder2:
    def __init__(
            self,
            *,
            tensor_map_out: TensorMap,
            filters_per_dense_block: List[int],
            conv_layer_type: str,
            conv_x: List[int],
            conv_y: List[int],
            conv_z: List[int],
            block_size: int,
            activation: str,
            normalization: str,
            regularization: str,
            regularization_rate: float,
            upsample_x: int,
            upsample_y: int,
            upsample_z: int,
    ):
        dimension = tensor_map_out.axes()
        self.dense_blocks = [
            DenseConvolutionalBlock(
                dimension=tensor_map_out.axes(), conv_layer_type=conv_layer_type, filters=filters, conv_x=[x]*block_size,
                conv_y=[y]*block_size, conv_z=[z]*block_size, block_size=block_size, activation=activation, normalization=normalization,
                regularization=regularization, regularization_rate=regularization_rate,
            )
            for filters, x, y, z in zip(filters_per_dense_block, conv_x, conv_y, conv_z)
        ]
        conv_layer, _ = _conv_layer_from_kind_and_dimension(dimension, 'conv', conv_x, conv_y, conv_z)
        self.conv_label = conv_layer(tensor_map_out.shape[-1], _one_by_n_kernel(dimension), activation=tensor_map_out.activation, name=tensor_map_out.output_name())
        self.upsamples = [_upsampler(dimension, upsample_x, upsample_y, upsample_z) for _ in range(len(filters_per_dense_block) + 1)]
        print(f'Decode has: {list(enumerate(zip(self.dense_blocks, self.upsamples)))}')
    def __call__(self, x: Tensor) -> Tensor:
        for i, (dense_block, upsample) in enumerate(zip(self.dense_blocks, self.upsamples)):
            
            x = upsample(x)
            x = dense_block(x)
        return self.conv_label(x)
    
    

In [ ]:
from tensorflow.keras.layers import Layer
from tensorflow import acos

def l2_norm(x, axis=None):
    """
    takes an input tensor and returns the l2 norm along specified axis
    """

    square_sum = K.sum(K.square(x), axis=axis, keepdims=True)
    norm = K.sqrt(K.maximum(square_sum, K.epsilon()))

    return norm

def pairwise_cosine_difference(t1, t2):
    """
    A [batch x n x d] tensor of n rows with d dimensions
    B [batch x m x d] tensor of n rows with d dimensions

    returns:
    D [batch x n x m] tensor of cosine similarity scores between each point i<n, j<m
    """
    t1_norm = t1 / l2_norm(t1, axis=-1)
    t2_norm = t2 / l2_norm(t2, axis=-1)
    dot = K.clip(K.batch_dot(t1, t2), -1, 1)
    return acos(dot)

class CosineLossLayer(Layer):
    """Layer that creates an Cosine loss."""
    def __init__(self, weight):
        super(CosineLossLayer, self).__init__()
        self.weight = weight
    def get_config(self):
        config = super().get_config().copy()
        config.update({'weight': self.weight})
        return config
    def call(self, inputs):
        # We use `add_loss` to create a regularization loss
        # that depends on the inputs.
        self.add_loss(self.weight * pairwise_cosine_difference(inputs[0], inputs[1]))
        return inputs

class L2LossLayer(Layer):
    """Layer that creates an L2 loss."""
    def __init__(self, weight):
        super(L2LossLayer, self).__init__()
        self.weight = weight
        
    def get_config(self):
        config = super().get_config().copy()
        config.update({'weight': self.weight})
        return config
    
    def call(self, inputs):
        self.add_loss(self.weight * tf.reduce_sum(tf.square(inputs[0] - inputs[1])))
        return inputs

In [ ]:
def make_paired_autoencoder_model(
    pairs: List[Tuple[TensorMap, TensorMap]],
    pair_loss = 'cosine',
    **kwargs
) -> Model:
    inputs = {tm: Input(shape=tm.shape, name=tm.input_name()) for tm in args.tensor_maps_in}
    original_outputs = {tm:1 for tm in args.tensor_maps_out}
    multimodal_activations = []
    outputs = []
    losses = []
    for left, right in pairs:
        args.tensor_maps_in = [left]
        left_model = make_multimodal_multitask_model(**args.__dict__)
        encode_left = make_hidden_layer_model(left_model, [left], args.hidden_layer)
        h_left = encode_left(inputs[left])
        
        args.tensor_maps_in = [right]
        right_model = make_multimodal_multitask_model(**args.__dict__)     
        encode_right = make_hidden_layer_model(right_model, [right], args.hidden_layer)
        h_right = encode_right(inputs[right])        
        
        if pair_loss == 'cosine':
            loss_layer = CosineLossLayer(1.0)
        elif pair_loss == 'euclid':
            loss_layer = L2LossLayer(1.0)
        
        paired_embeddings = loss_layer([h_left, h_right])
        multimodal_activations.extend(paired_embeddings)
        
    multimodal_activation = Concatenate()(multimodal_activations)
    
    pre_decoder_shapes: Dict[TensorMap, Optional[Tuple[int, ...]]] = {}
    for tm in args.tensor_maps_out:
        shape = _calc_start_shape(num_upsamples=len(args.dense_blocks), output_shape=tm.shape, 
                                  upsample_rates=[args.pool_x, args.pool_y, args.pool_z], 
                                  channels=args.dense_blocks[-1])    
        
        restructure = FlatToStructure(output_shape=shape, activation=args.activation, 
                                      normalization=args.dense_normalize)
        
        decode = ConvDecoder2(
            tensor_map_out=tm,
            filters_per_dense_block=args.dense_blocks[::-1],
            conv_layer_type=args.conv_type,
            conv_x=args.conv_x,
            conv_y=args.conv_y,
            conv_z=args.conv_z,
            block_size=args.block_size,
            activation=args.activation,
            normalization=args.conv_normalize,
            regularization=args.conv_regularize,
            regularization_rate=args.conv_regularize_rate,
            upsample_x=args.pool_x,
            upsample_y=args.pool_y,
            upsample_z=args.pool_z,
        )
        
        outputs.append(decode(restructure(multimodal_activation)))
        losses.append(tm.loss)

    args.tensor_maps_out =  list(original_outputs.keys())
    args.tensor_maps_in = list(inputs.keys())
    
    m = Model(inputs=list(inputs.values()), outputs=outputs)
    my_metrics = {tm.output_name(): tm.metrics for tm in args.tensor_maps_out}
    opt = Adam(lr=kwargs['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    m.compile(optimizer=opt, loss=losses, metrics=my_metrics)
    m.summary()
    
    if kwargs['model_layers'] is not None:
        m.load_weights(kwargs['model_layers'], by_name=True)
        print(f"Loaded model weights from:{kwargs['model_layers']}")
        
    return m

In [ ]:
sys.argv = ['train', 
            '--tensors', '/mnt/disks/sax-lax-40k-lvm/2020-01-29/', 
            '--input_tensors', 'ecg.ecg_rest','mri.lax_4ch_diastole_slice0_224_3d', 'mri.cine_segmented_lax_4ch_diastole', 
            '--output_tensors', 'ecg.ecg_rest','mri.lax_4ch_diastole_slice0_224_3d', 'mri.cine_segmented_lax_4ch_diastole',
            '--activation', 'swish',
            '--conv_layers', '32',
            '--conv_x', '9', '9', '9',
            '--conv_y', '3', '3', '3', 
            '--conv_z', '3', '3', '3',
            '--dense_blocks', '32', '32', '32',
            '--block_size', '3',
            '--dense_layers', '256',
            '--pool_x', '2',
            '--pool_y', '2',
            '--batch_size', '1',
            '--patience', '44',
            '--epochs', '496',
            '--learning_rate', '0.0001',
            '--training_steps', '128',
            '--validation_steps', '30',
            '--test_steps', '8',
            '--num_workers', '4',
            '--inspect_model',
            '--tensormap_prefix', 'ml4h.tensormap.ukb',
            '--id', 'ecg_mri_lax_4ch_diastole_euclid_triplet_segmenter_256d']
args = parse_args()
pairs = [
    (args.tensor_maps_in[0], args.tensor_maps_in[1]),
    (args.tensor_maps_in[1], args.tensor_maps_in[2])
]
overparameterized_model = make_paired_autoencoder_model(pairs, pair_loss='euclid', **args.__dict__)
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(**args.__dict__)
train_model_from_generators(
        overparameterized_model, generate_train, generate_valid, args.training_steps, args.validation_steps, args.batch_size,
        args.epochs, args.patience, args.output_folder, args.id, args.inspect_model, args.inspect_show_labels,
)

In [ ]:
out_path = os.path.join(args.output_folder, args.id + '/')
test_data, test_labels, test_paths = big_batch_from_minibatch_generator(generate_test, args.test_steps)
print(list(test_data.keys()))

preds = overparameterized_model.predict(test_data)
print([p.shape for p in preds])
print([tm.name for tm in args.tensor_maps_out])
print(test_paths)

In [ ]:
from ml4h.plots import _plot_reconstruction
_plot_reconstruction(args.tensor_maps_out[0],  test_data['input_strip_continuous'], preds[0], out_path, test_paths)

In [ ]:
from ml4h.explorations import predictions_to_pngs
predictions_to_pngs(preds, args.tensor_maps_in, args.tensor_maps_out, 
                    test_data, test_labels, test_paths, out_path)


In [ ]:
print(list(test_data['input_strip_continuous'].shape))

In [ ]:
test_data, test_labels, test_paths = big_batch_from_minibatch_generator(generate_test, args.test_steps)
test_data['input_strip_continuous'] = np.random.random((args.test_steps*args.batch_size, 5000, 12))
ecg_noise_preds = overparameterized_model.predict(test_data)
out_path = os.path.join(args.output_folder, args.id + '/ecg_noise/')
if not os.path.exists(os.path.dirname(out_path)):
    os.makedirs(os.path.dirname(out_path))
_plot_reconstruction(args.tensor_maps_out[0],  test_data['input_strip_continuous'], ecg_noise_preds[0], out_path, test_paths)
predictions_to_pngs(ecg_noise_preds, args.tensor_maps_in, args.tensor_maps_out, 
                    test_data, test_labels, test_paths, out_path)

In [ ]:
def plot_ae_towards_attractor(model, test_data, test_labels, test_key, test_index, rows=4, samples=6, steps = 10):
    sample_every = steps//samples
    fig, axes = plt.subplots(rows, samples, figsize=(samples*16,rows*16))
    col = 0
    original = test_data[test_key].copy()
    for i in range(steps):
        if i % sample_every == 0 and col < samples:
            for j in range(rows):
                if len(test_data[test_key].shape) == 4:
                    axes[j, col].imshow(test_data[test_key][j, :, :, 0], cmap = 'gray')
                    axes[j, col].set_yticks(())
                elif len(test_data[test_key].shape) == 3:
                    for l in range(12):
                        axes[j, col].plot(range(1200), test_data[test_key][j, :1200, l])
                        axes[j, col].set_yticks(())
            col += 1
        preds = model.predict(test_data) 
        test_data[test_key] = preds[test_index]
    test_data[test_key] = original
    plt.show()
    return preds

In [ ]:
# IPython imports
%matplotlib inline
import matplotlib.pyplot as plt


test_data, test_labels, test_paths = big_batch_from_minibatch_generator(generate_test, args.test_steps)
test_key = 'input_lax_4ch_diastole_slice0_224_3d_continuous'
test_shape = test_data[test_key].shape
test_data[test_key] = np.random.random(test_shape)
out_path = os.path.join(args.output_folder, args.id, test_key + '_noise/')
if not os.path.exists(os.path.dirname(out_path)):
    os.makedirs(os.path.dirname(out_path))
noise_preds = plot_ae_towards_attractor(overparameterized_model, test_data, test_labels, test_key, 
                                        test_index=1, rows=8, samples=4, steps = 18)
print(list(test_data.keys()))
_plot_reconstruction(args.tensor_maps_out[0],  test_data['input_strip_continuous'], 
                     noise_preds[0], out_path, test_paths)
predictions_to_pngs(noise_preds, args.tensor_maps_in, args.tensor_maps_out, 
                    test_data, test_labels, test_paths, out_path)

In [ ]:
# IPython imports
%matplotlib inline
import matplotlib.pyplot as plt


test_data, test_labels, test_paths = big_batch_from_minibatch_generator(generate_test, args.test_steps)
test_key = 'input_strip_continuous'
test_shape = test_data[test_key].shape
test_data[test_key] = np.random.random(test_shape)
out_path = os.path.join(args.output_folder, args.id, test_key + '_noise/')
if not os.path.exists(os.path.dirname(out_path)):
    os.makedirs(os.path.dirname(out_path))
noise_preds = plot_ae_towards_attractor(overparameterized_model, test_data, test_labels, test_key, 
                                        test_index=0, rows=6, samples=6, steps = 24)
print(list(test_data.keys()))
_plot_reconstruction(args.tensor_maps_out[0],  test_data['input_strip_continuous'], 
                     noise_preds[0], out_path, test_paths)
predictions_to_pngs(noise_preds, args.tensor_maps_in, args.tensor_maps_out, 
                    test_data, test_labels, test_paths, out_path)

In [ ]:
df = pd.read_csv('/home/sam/ml/trained_models/lax_4ch_diastole_autoencode_leaky_converge/tensors_all_union.csv')
df['21003_Age-when-attended-assessment-centre_2_0'].plot.hist(bins=30)
hidden_inference = './recipes_output/ecg_mri_lax_4ch_diastole_paired_autoencoder_2blocks_256d_200_samples/hidden_inference_ecg_mri_lax_4ch_diastole_paired_autoencoder_2blocks_256d_200_samples.tsv'


df2 = pd.read_csv(hidden_inference, sep='\t')
df['fpath'] = pd.to_numeric(df['fpath'], errors='coerce')
df2['sample_id'] = pd.to_numeric(df2['sample_id'], errors='coerce')
#df.info()
latent_df = pd.merge(df, df2, left_on='fpath', right_on='sample_id', how='inner')
#latent_df.info()
df3 = pd.read_csv('/home/sam/tsvs/ttn_disease.tsv', sep='\t')
df4 = pd.read_csv('/home/sam/csvs/has_exome.csv')
latent_df = pd.merge(df3, latent_df, left_on='sample_id', right_on='sample_id', how='right')
latent_df.info()
print(latent_df['has_ttntv'].value_counts())

In [ ]:
def pca_on_matrix(matrix, pca_components):
    pca = PCA()
    pca.fit(matrix)
    print(f'PCA explains {100*np.sum(pca.explained_variance_ratio_[:pca_components]):0.1f}% of variance with {pca_components} top PCA components.')
    matrix_reduced = pca.transform(matrix)[:, :pca_components]
    print(f'PCA reduces matrix shape:{matrix_reduced.shape} from matrix shape: {matrix.shape}')
    plot_scree(pca_components, 100*pca.explained_variance_ratio_)
    return pca, matrix_reduced

def plot_scree(pca_components, percent_explained):
    _ = plt.figure(figsize=(6, 4))
    plt.plot(range(len(percent_explained)), percent_explained, 'g.-', linewidth=1)
    plt.axvline(x=pca_components, c='r', linewidth=3)
    label = f'{np.sum(percent_explained[:pca_components]):0.1f}% of variance explained by top {pca_components} of {len(percent_explained)} components'
    plt.text(pca_components+0.02*len(percent_explained), percent_explained[1], label)
    plt.title('Scree Plot')
    plt.xlabel('Principal Components')
    plt.ylabel('% of Variance Explained by Each Component')
    figure_path = f'./results/pca_{pca_components}_of_{len(percent_explained)}_testimonials.png'
    if not os.path.exists(os.path.dirname(figure_path)):
        os.makedirs(os.path.dirname(figure_path))
    plt.savefig(figure_path)
    
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)) * 180 / 3.141592

def directions_in_latent_space(stratify_column, stratify_thresh, split_column, split_thresh, latent_cols, latent_df):
    hit = latent_df.loc[latent_df[stratify_column] >= stratify_thresh][latent_cols].to_numpy()
    miss = latent_df.loc[latent_df[stratify_column] < stratify_thresh][latent_cols].to_numpy()
    miss_mean_vector = np.mean(miss, axis=0)
    hit_mean_vector = np.mean(hit, axis=0)
    strat_vector = hit_mean_vector - miss_mean_vector
    
    hit1 = latent_df.loc[(latent_df[stratify_column] >= stratify_thresh) 
                        & (latent_df[split_column] >= split_thresh)][latent_cols].to_numpy()
    miss1 = latent_df.loc[(latent_df[stratify_column] < stratify_thresh) 
                        & (latent_df[split_column] >= split_thresh)][latent_cols].to_numpy()
    hit2 = latent_df.loc[(latent_df[stratify_column] >= stratify_thresh) 
                        & (latent_df[split_column] < split_thresh)][latent_cols].to_numpy()
    miss2 = latent_df.loc[(latent_df[stratify_column] < stratify_thresh) 
                        & (latent_df[split_column] < split_thresh)][latent_cols].to_numpy()
    miss_mean_vector1 = np.mean(miss1, axis=0)
    hit_mean_vector1 = np.mean(hit1, axis=0)
    angle1 = angle_between(miss_mean_vector1, hit_mean_vector1)
    miss_mean_vector2 = np.mean(miss2, axis=0)
    hit_mean_vector2 = np.mean(hit2, axis=0)
    angle2 = angle_between(miss_mean_vector2, hit_mean_vector2)
    h1_vector = hit_mean_vector1-miss_mean_vector1
    h2_vector = hit_mean_vector2-miss_mean_vector2
    angle3 = angle_between(h1_vector, h2_vector)
    angle4 = angle_between(strat_vector, h1_vector)
    angle5 = angle_between(strat_vector, h2_vector)
    print(f'\n Between {stratify_column}, and splits: {split_column}\n',
          f'Angles h1 and m1: {angle1:.2f}, h2 and m2 {angle2:.2f} h1-m1 and h2-m2 {angle3:.2f} degrees.\n'
          f'stratify threshold: {stratify_thresh}, split thresh: {split_thresh}, \n'
          f'hit_mean_vector2 shape {miss_mean_vector1.shape}, miss1:{hit_mean_vector2.shape} \n'
          f'Hit1 shape {hit1.shape}, miss1:{miss1.shape} threshold:{stratify_thresh}\n'
          f'Hit2 shape {hit2.shape}, miss2:{miss2.shape}\n')

def stratify_latent_space(stratify_column, stratify_thresh, latent_cols, latent_df):
    hit = latent_df.loc[latent_df[stratify_column] >= stratify_thresh][latent_cols].to_numpy()
    miss = latent_df.loc[latent_df[stratify_column] < stratify_thresh][latent_cols].to_numpy()
    miss_mean_vector = np.mean(miss, axis=0)
    hit_mean_vector = np.mean(hit, axis=0)
    angle = angle_between(miss_mean_vector, hit_mean_vector)
    print(f'Angle between {stratify_column} centroid and all others is: {angle:.1f} degrees.\n'
          f'Hit shape {hit.shape}, miss:{miss.shape} threshold:{stratify_thresh}\n'
          f'Distance: {np.linalg.norm(hit_mean_vector-miss_mean_vector):.3f}, Hit std {np.std(hit, axis=1).mean():.3f}, miss std:{np.std(miss, axis=1).mean():.3f}\n')
    
def plot_pcs(sides, color_key):
    f, axes = plt.subplots(sides, sides, figsize=(16, 16))
    for i, ax in enumerate(axes.ravel()):
        colors = latent_df[color_key].to_numpy()
        points = ax.scatter(matrix_reduce[:, i], matrix_reduce[:, i+1], c=colors)
        f.colorbar(points, ax=ax)
        



In [ ]:
import matplotlib.pyplot as plt
latent_dimension = 256
latent_cols = [f'latent_{i}' for i in range(latent_dimension)]
pca, matrix_reduce = pca_on_matrix(df2[latent_cols].to_numpy(), 10)
for strat in ['Sex_Female_0_0', 'has_ttntv', 'atrial_fibrillation_or_flutter', 
              'coronary_artery_disease', 'hypertension']:
    stratify_latent_space(strat, 1.0, latent_cols, latent_df)
strats = ['LVEF', 'LVM', 'LVEDV', 'sample_id',
              '21001_Body-mass-index-BMI_0_0', '21003_Age-when-attended-assessment-centre_2_0']
theshes = [45, 100, 150, 3500000, 27.5, 70]
for strat, thresh in zip(strats, theshes):
    stratify_latent_space(strat, thresh, latent_cols, latent_df)

In [ ]:
import matplotlib.pyplot as plt
latent_dimension = 256
latent_cols = [f'latent_{256+i}' for i in range(latent_dimension)]
pca, matrix_reduce = pca_on_matrix(df2[latent_cols].to_numpy(), 10)
for strat in ['Sex_Female_0_0', 'has_ttntv', 'atrial_fibrillation_or_flutter', 
              'coronary_artery_disease', 'hypertension']:
    stratify_latent_space(strat, 1.0, latent_cols, latent_df)
strats = ['LVEF', 'LVM', 'LVEDV', 'sample_id',
              '21001_Body-mass-index-BMI_0_0', '21003_Age-when-attended-assessment-centre_2_0']
theshes = [45, 100, 150, 3500000, 27.5, 70]
for strat, thresh in zip(strats, theshes):
    stratify_latent_space(strat, thresh, latent_cols, latent_df)

In [ ]:
latent_dimension = 512
latent_cols = [f'latent_{i}' for i in range(latent_dimension)]
pca, matrix_reduce = pca_on_matrix(df2[latent_cols].to_numpy(), 10)
for strat in ['Sex_Female_0_0', 'atrial_fibrillation_or_flutter', 
              'coronary_artery_disease', 'hypertension']:
    stratify_latent_space(strat, 1.0, latent_cols, latent_df)
strats = ['LVEF', 'LVM', 'LVEDV', 'sample_id',
              '21001_Body-mass-index-BMI_0_0', '21003_Age-when-attended-assessment-centre_2_0']
theshes = [45, 100, 150, 3500000, 27.5, 70]
for strat, thresh in zip(strats, theshes):
    stratify_latent_space(strat, thresh, latent_cols, latent_df)

In [ ]:
latent_dimension = 256
latent_cols = [f'latent_{256+i}' for i in range(latent_dimension)]
pca, matrix_reduce = pca_on_matrix(df2[latent_cols].to_numpy(), 10)
c_strats = [ 'Sex_Female_0_0']
for c_strat in c_strats:
    strats = ['LVEF', 'LVM', 'LVEDV', 'sample_id',
                  '21001_Body-mass-index-BMI_0_0', '21003_Age-when-attended-assessment-centre_2_0']
    theshes = [50, 100, 150, 3750000, 27.5, 70]
    for strat, thresh in zip(strats, theshes):
        directions_in_latent_space(c_strat, 1.0, strat, thresh, latent_cols, latent_df)

In [ ]:
latent_dimension = 256
latent_cols = [f'latent_{256+i}' for i in range(latent_dimension)]
pca, matrix_reduce = pca_on_matrix(df2[latent_cols].to_numpy(), 10)
c_strats = [ 'sample_id']
for c_strat in c_strats:
    strats = ['LVEF', 'LVM', 'LVEDV', 'sample_id',
                  '21001_Body-mass-index-BMI_0_0', '21003_Age-when-attended-assessment-centre_2_0']
    theshes = [50, 100, 150, 3750000, 27.5, 70]
    for strat, thresh in zip(strats, theshes):
        directions_in_latent_space(c_strat, 3750000.0, strat, thresh, latent_cols, latent_df)

In [ ]:
latent_dimension = 256
latent_cols = [f'latent_{i}' for i in range(latent_dimension)]
ecg_encode = latent_df[latent_cols].to_numpy()
latent_cols = [f'latent_{256+i}' for i in range(latent_dimension)]
mri_encode = latent_df[latent_cols].to_numpy()
diff = np.sqrt(np.einsum('ij, ij->ij', ecg_encode - mri_encode, ecg_encode - mri_encode))
print(diff.shape) 
print(np.mean(diff))

In [ ]:
print(f'{ecg_encode[:5,:5]} \n{mri_encode[:5,:5]}')

In [ ]:
latent_dimension = 256
latent_cols = [f'latent_{i}' for i in range(latent_dimension)]
ecg_encode = latent_df[latent_cols].to_numpy()
latent_cols = [f'latent_{18+i}' for i in range(latent_dimension)]
mri_encode = latent_df[latent_cols].to_numpy()
diff = np.sqrt(np.einsum('ij, ij->ij', ecg_encode - mri_encode, ecg_encode - mri_encode))
print(diff.shape) 
print(np.mean(diff))

In [ ]:
ch2_random = np.random.random((4452, 256))
ch3_random = np.random.random((4452, 256))
diff = np.sqrt(np.einsum('ij, ij->ij', ch2_random - ch3_random, ch2_random - ch3_random))
print(diff.shape) 
print(np.mean(diff))

In [ ]:
sys.argv = ['train', 
            '--tensors', '/mnt/disks/sax-lax-40k-lvm/2020-01-29/', 
            '--input_tensors', 'ecg.ecg_rest', 'mri.lax_4ch_diastole_slice0_224_3d', 
            '--output_tensors', 'ecg.ecg_rest', 'mri.lax_4ch_diastole_slice0_224_3d',
            '--activation', 'swish',
            '--conv_layers', '32',
            '--conv_x', '9', '9', '9',
            '--conv_y', '3', '3', '3', 
            '--conv_z', '3', '3', '3', 
            '--dense_blocks', '32', '32', '32',
            '--block_size', '3',
            '--dense_layers', '256',
            '--pool_x', '2',
            '--pool_y', '2',
            '--batch_size', '1',
            '--patience', '44',
            '--epochs', '532',
            '--learning_rate', '0.0002',
            '--training_steps', '72',
            '--validation_steps', '30',
            '--test_steps', '8',
            '--num_workers', '4',
            '--inspect_model',
            '--tensormap_prefix', 'ml4h.tensormap.ukb',
            '--hidden_layer', 'concatenate_36',
            '--model_file', './recipes_output/ecg_mri_lax_4ch_diastole_paired_autoencoder_2blocks_256d_200_samples/ecg_mri_lax_4ch_diastole_paired_autoencoder_2blocks_256d_200_samples.h5',
            '--train_csv', '/home/sam/lvh/small_set.csv',
            #'--sample_csv', '/home/sam/lvh/lvh_hold_out.txt',
            '--id', 'ecg_mri_lax_4ch_diastole_paired_autoencoder_2blocks_256d_200_samples']
args = parse_args()

#overparameterized_model = make_multimodal_multitask_model(**args.__dict__)
#infer_hidden_layer_multimodal_multitask(args)
#generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(**args.__dict__)
# train_model_from_generators(
#         overparameterized_model, generate_train, generate_valid, args.training_steps, args.validation_steps, args.batch_size,
#         args.epochs, args.patience, args.output_folder, args.id, args.inspect_model, args.inspect_show_labels,
# )